In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy import stats
from scipy.integrate import quad
import pandas as pd
from scipy.stats import norm

# Case 5: Multi-Look and American Options

## Part 1: Asian Options

In [22]:
S0 = 100 
r = 0.02 
sigma = 0.15
T = 10  # Time horizon
n = 10  # Number of observations
n_sims = 100000  # Number of simulation paths
guarantee = 100
K = 100

In [17]:
# Exercise 1.1
def AsianOptionMonteCarlo(S0, T, r, sigma, numSteps, numSim):
    dt = T / numSteps 

    # Stock price paths
    S = np.zeros((numSim, numSteps + 1))  
    S[:, 0] = S0  

    Z = np.random.normal(0, 1, (numSim, numSteps)) 
    for t in range(1, numSteps + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1])

    # The new payoff function: max(average stock price - 100, 0)
    average_price = np.mean(S[:, 1:], axis=1)  
    payoff = np.maximum(average_price - 100, 0)

    discountedPrice = np.exp(-r * T) * np.mean(payoff)
    SE = np.exp(-r * T) * np.std(payoff) / np.sqrt(numSim)

    return discountedPrice, SE

asianOption_price, SE = AsianOptionMonteCarlo(S0, T, r, sigma, numSteps=10, numSim=10000)

print("Price of the Asian contract (Monte Carlo): ", asianOption_price, "±", SE)

Price of the Asian contract (Monte Carlo):  15.674115212225907 ± 0.2240626755242127


In [39]:
# Exercise 1.2
def blackScholes(S0, r, sigma, T, K):

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    phi = norm.cdf
    optionPrice = S0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return optionPrice

print("Black-Scholes value of European call-option: ",blackScholes(S0, r, sigma, T=10, K=100))

Black-Scholes value of European call-option:  27.571349248747218


In [ ]:
# Exercise 1.3
def deriveMoments(S0, r, sigma, T, N):   
    
    EQ_A = (1 / N) * sum(S0 * np.exp(r * np.arange(1, N + 1)))
    
    EQ_A2 = (1 / N**2) * sum(
        sum(S0**2 * np.exp(r * (i + np.arange(1, N + 1)) + sigma**2 * np.minimum(i, np.arange(1, N + 1))))
        for i in range(1, N + 1)
    )
    
    A0 = EQ_A / np.exp(r * T)
    b = np.sqrt((1 / T) * np.log(EQ_A2 / (A0**2)) - 2 * r)
    
    return A0, b

def asianOption_closedForm(S0, r, sigma, T, K, N):
    
    A0, b = deriveMoments(S0, r, sigma, T, N)
    print("A0: ", A0)
    print("b: ", b)

    d1 = (np.log(A0 / K) + (r + 0.5 * b**2) * T) / (b * np.sqrt(T))
    d2 = (np.log(A0 / K) + (r - 0.5 * b**2) * T) / (b * np.sqrt(T))
    
    phi = norm.cdf
    optionPrice = A0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return optionPrice

print("Closed-form price of Asian option",asianOption_closedForm(S0, r, sigma, T, K, N = 10))

A0:  91.54399082959372
b:  0.09584030077451944
Closed-form price of Asian option 15.973823843043952


## Part 2: Unit-linked with continuous guarantee

**Remark:** We now have a continous guarantee, where the client has the right to "early exercise" the guarantee and walk away with value of 100 at any time 0 < t < 10

In [40]:
def binomialTreeContinuous(S0, guarantee, r, n, T, sigma):

    dt = T / n  
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(-sigma * np.sqrt(dt))

    p = (R_dt - d) / (u - d)

    # Stock prices 
    stock_prices = np.array([guarantee * (u ** j) * (d ** (n - j)) for j in range(n + 1)])
    
    # Payoff 
    contract_values = np.maximum(stock_prices, guarantee)
    
    # Backward induction 
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            continuation_value = (p * contract_values[j + 1] + (1 - p) * contract_values[j]) * np.exp(-r * dt)
            contract_values[j] = max(guarantee, continuation_value)  

    return contract_values[0]

print("Price of unit-linked with continous guarantee",binomialTreeContinuous(S0, guarantee, r, n, T, sigma))

Price of unit-linked with continous guarantee 111.84198439352726


In [ ]:
def unit_linked_payoff(fund_value, guarantee):
    """
    Payoff function for a unit-linked contract with a continuous guarantee.
    """
    return np.maximum(fund_value, guarantee)

def least_squares_monte_carlo_unitlinked_continuous(S0,guarantee, r, sigma, T, n, n_sims):
    dt = T / n  
    discount_factor = np.exp(-r * dt)  

    # Simulate fund value paths
    fund_paths = np.zeros((n_sims, n + 1))
    fund_paths[:, 0] = S0
    for t in range(1, n + 1):
        z = np.random.standard_normal(n_sims)
        fund_paths[:, t] = fund_paths[:, t - 1] * np.exp(
            (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z
        )

    # Initialize cash flows at maturity
    cash_flows = unit_linked_payoff(fund_paths[:, -1], guarantee)

    # Work backward through the time steps
    for t in range(n - 1, 0, -1):
        in_the_money = unit_linked_payoff(fund_paths[:, t], guarantee) > 0
        if np.sum(in_the_money) == 0:
            continue  

        # Regression 
        X = fund_paths[in_the_money, t]
        Y = cash_flows[in_the_money] * discount_factor
        regression = np.polyfit(X, Y, 2)  # quadratic polynomial
        continuation_values = np.polyval(regression, X)

        # Comparison
        surrender_values = unit_linked_payoff(X, guarantee)
        exercise = surrender_values > continuation_values

        # Update cash flows based on exercise decision
        cash_flows[in_the_money] = np.where(
            exercise, surrender_values, cash_flows[in_the_money] * discount_factor
        )

    # Discount cash flows
    contract_value = np.mean(cash_flows) * np.exp(-r * dt)
    SE = np.exp(-r * T) * np.std(surrender_values) / np.sqrt(n_sims)
    
    return contract_value, SE



S0 = 100          
r = 0.02          
sigma = 0.15      
T = 10            
n = 10            
n_sims = 100000   
guarantee = 100  

contract_value, SE = least_squares_monte_carlo_unitlinked_continuous(S0, guarantee, r, sigma, T, n, n_sims)
print("The value of the unit-linked contract with continuous guarantee is: ", contract_value, "±", SE)


The value of the unit-linked contract with continuous guarantee is:  111.36658440369438 ± 0.02689632202126108


In [14]:
def simulate_paths(S0, r, sigma, T, M, N):
    dt = T / M
    paths = np.zeros((N, M + 1))
    paths[:, 0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(N)
        paths[:, t] = paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    return paths

paths = simulate_paths(S0, r, sigma, T, M=100000, N=10)

# LSMC valuation
def least_squares_monte_carlo(paths, r, G, T, M):
    dt = T / M
    df = np.exp(-r * dt)  # Discount factor per time step
    cashflows = np.maximum(paths[:, -1], G)  # Payoff at maturity
    
    for t in range(M - 1, 0, -1):
        # In-the-money paths
        itm = paths[:, t] < G  # In-the-money condition (exercise depends on guarantee)
        X = paths[itm, t].reshape(-1, 1)
        Y = cashflows[itm] * df
        
        if len(X) > 0:  # Avoid empty regression cases
            # Regression to estimate continuation value
            reg = LinearRegression().fit(X, Y)
            continuation_value = reg.predict(X)
            
            # Decide whether to exercise or continue
            exercise_value = G - X.flatten()
            exercise = exercise_value > continuation_value
            cashflows[itm] = np.where(exercise, G, cashflows[itm] * df)
        cashflows[~itm] *= df  # Update continuation value for out-of-the-money paths

    # Discount back to present value
    option_value = np.mean(cashflows) * np.exp(-r * dt)
    return option_value

least_squares_monte_carlo(paths, r, 100, T, M=100000)

NameError: name 'LinearRegression' is not defined